In [1]:
# %pip install scikit-learn pandas joblib

# Operation 1

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

data = pd.read_csv('pose_data.csv') 

X = data.drop(columns=['Label'])  
y = data['Label']                

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))

joblib.dump(model, 'activity_classifier.pkl')


Classification Report:
                    precision    recall  f1-score   support

            cough       1.00      0.80      0.89         5
         coughing       0.00      0.00      0.00         6
          falling       0.83      0.71      0.77        14
         headache       0.72      0.95      0.82        19
     heart stroke       0.79      0.68      0.73        22
          sitting       0.75      0.94      0.83        16
         standing       1.00      0.71      0.83         7
standing normally       1.00      1.00      1.00         8

         accuracy                           0.77        97
        macro avg       0.76      0.72      0.73        97
     weighted avg       0.77      0.77      0.76        97



['activity_classifier.pkl']

# Operation 2

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

joblib.dump(scaler, 'scaler.pkl')


['scaler.pkl']

In [ ]:
# import cv2
# import mediapipe as mp
# import numpy as np
# import joblib
# import time

# model = joblib.load('activity_classifier.pkl')
# scaler = joblib.load('scaler.pkl')

# mp_pose = mp.solutions.pose
# mp_drawing = mp.solutions.drawing_utils

# def predict_activity(landmarks):
#     feature_vector = []
#     for landmark in landmarks:
#         feature_vector.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
    
#     feature_vector = np.array(feature_vector).reshape(1, -1)
#     feature_vector = scaler.transform(feature_vector)
    
#     activity = model.predict(feature_vector)
#     return activity[0]

# cap = cv2.VideoCapture(0)

# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             print("Ignoring empty frame.")
#             break

#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False

#         results = pose.process(image)

#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         if results.pose_landmarks:
#             mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
#             landmarks = results.pose_landmarks.landmark
#             activity = predict_activity(landmarks)
#         else:
#             activity = "No Person Detected"

#         cv2.putText(image, f"Activity: {activity}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

#         cv2.imshow('Real-time Activity Detection', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

# cap.release()
# cv2.destroyAllWindows()


## DEEP LEARNING

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import joblib

# Load the dataset
data = pd.read_csv('pose_data.csv')

# Separate features and labels
X = data.drop(columns=['Label'])
y = data['Label']

# Encode labels into integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Save label encoder for future use
joblib.dump(label_encoder, 'label_encoder.pkl')

# Standardize the feature data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Save the scaler for future use
joblib.dump(scaler, 'scaler.pkl')

# One-hot encode the labels for deep learning
y_one_hot = to_categorical(y_encoded)

# Train-test split (using stratify for balanced classes)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_one_hot, test_size=0.2, random_state=42, stratify=y_one_hot
)

# Build the deep learning model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_one_hot.shape[1], activation='softmax')  # Output layer with softmax for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot encoded test labels back to integers

# Handle the classification report
labels = np.unique(y_test_labels)
print("Classification Report:\n", classification_report(
    y_test_labels,
    y_pred,
    labels=labels,
    target_names=label_encoder.inverse_transform(labels)
))

# Save the trained model
model.save('activity_classifier_deep.h5')


# TEST 2


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
import joblib

# Load the dataset
data = pd.read_csv('pose_data.csv')

# Separate features and labels
X = data.drop(columns=['Label'])
y = data['Label']

# Encode labels into integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Save label encoder for future use
joblib.dump(label_encoder, 'label_encoder.pkl')

# Standardize the feature data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Save the scaler for future use
joblib.dump(scaler, 'scaler.pkl')

# One-hot encode the labels for deep learning
y_one_hot = to_categorical(y_encoded)

# Train-test split (using stratify for balanced classes)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_one_hot, test_size=0.2, random_state=42, stratify=y_one_hot
)

# Build the deep learning model
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(y_one_hot.shape[1], activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for learning rate adjustment and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=500,
    batch_size=64,
    validation_split=0.2,
    callbacks=[reduce_lr, early_stop]
)

# Evaluate the model
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot encoded test labels back to integers

# Classification report
labels = np.unique(y_test_labels)
print("Classification Report:\n", classification_report(
    y_test_labels,
    y_pred,
    labels=labels,
    target_names=label_encoder.inverse_transform(labels)
))

# Save the trained model
model.save('activity_classifier_optimized.h5')

print("Model and preprocessing artifacts saved successfully!")


c:\anacondadata\envs\human\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 240ms/step - accuracy: 0.1451 - loss: 3.0752 - val_accuracy: 0.3500 - val_loss: 2.1569 - learning_rate: 0.0010
Epoch 2/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4415 - loss: 2.0989 - val_accuracy: 0.5500 - val_loss: 1.8473 - learning_rate: 0.0010
Epoch 3/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5813 - loss: 1.6968 - val_accuracy: 0.6000 - val_loss: 1.6127 - learning_rate: 0.0010
Epoch 4/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.6609 - loss: 1.3404 - val_accuracy: 0.7000 - val_loss: 1.4192 - learning_rate: 0.0010
Epoch 5/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.7280 - loss: 1.2430 - val_accuracy: 0.7500 - val_loss: 1.2595 - learning_rate: 0.0010
Epoch 6/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.8780 - loss: 0.9105 - val_accuracy: 0.9250 - val_loss: 1.1328 - learning_rate: 0.0010
Epoch 7/500
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8670 - loss: 0.8274 - val_accuracy:

Model and preprocessing artifacts saved successfully!


# Cam TEst 2

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import tensorflow as tf

# Load the trained model, scaler, and label encoder
model = tf.keras.models.load_model('activity_classifier_optimized.h5')
scaler = joblib.load('scaler.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to preprocess pose landmarks
def preprocess_landmarks(landmarks):
    try:
        feature_vector = [
            [landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in landmarks
        ]
        feature_vector = np.array(feature_vector).flatten().reshape(1, -1)
        feature_vector = scaler.transform(feature_vector)  # Standardize the feature vector
        return feature_vector
    except Exception as e:
        print(f"Error during preprocessing: {e}")
        return None

# Function to predict activity from processed landmarks
def predict_activity(processed_landmarks):
    try:
        predictions = model.predict(processed_landmarks)
        predicted_class = np.argmax(predictions, axis=1)[0]
        return label_encoder.inverse_transform([predicted_class])[0]
    except Exception as e:
        print(f"Error during prediction: {e}")
        return "Prediction Error"

# Start webcam feed
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Pose estimation
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No frame received from webcam. Exiting.")
            break

        # Convert image to RGB for Mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose landmarks
        results = pose.process(image)

        # Convert back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # If pose landmarks are detected
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            landmarks = results.pose_landmarks.landmark

            # Preprocess landmarks and predict activity
            processed_landmarks = preprocess_landmarks(landmarks)
            activity = (
                predict_activity(processed_landmarks)
                if processed_landmarks is not None
                else "Processing Error"
            )
        else:
            activity = "No Person Detected"

        # Display the activity label on the video feed
        cv2.putText(image, f"Activity: {activity}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Show the video feed
        cv2.imshow('Real-time Activity Detection', image)

        # Quit with 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            print("Exiting application.")
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


# TEST 3

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import joblib

# Load the dataset
data = pd.read_csv('pose_data.csv')

# Separate features and labels
X = data.drop(columns=['Label'])
y = data['Label']

# Encode labels into integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Save label encoder for future use
joblib.dump(label_encoder, 'label_encoder.pkl')

# Standardize the feature data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Save the scaler for future use
joblib.dump(scaler, 'scaler.pkl')

# One-hot encode the labels for deep learning
y_one_hot = to_categorical(y_encoded)

# Train-test split (using stratify for balanced classes)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_one_hot, test_size=0.2, random_state=42, stratify=y_encoded
)

# Build the deep learning model
model = Sequential([
    Dense(512, activation='relu', kernel_regularizer=l2(0.001), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(y_one_hot.shape[1], activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for learning rate adjustment and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=64,
    validation_split=0.2,
    callbacks=[reduce_lr, early_stop]
)

# Evaluate the model
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot encoded test labels back to integers

# Classification report
labels = np.unique(y_test_labels)
print("Classification Report:\n", classification_report(
    y_test_labels,
    y_pred,
    labels=labels,
    target_names=label_encoder.inverse_transform(labels)
))

# Confusion matrix
conf_matrix = confusion_matrix(y_test_labels, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Plot training and validation accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Save the trained model
model.save('activity_classifier_optimized.h5')

print("Model and preprocessing artifacts saved successfully!")


# cam test 3

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import joblib
import mediapipe as mp

# Load the trained model, scaler, and label encoder
model = tf.keras.models.load_model('activity_classifier_optimized.h5')
scaler = joblib.load('scaler.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Initialize Mediapipe Pose for pose detection
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to preprocess pose landmarks
def preprocess_landmarks(landmarks):
    try:
        # Extract features (x, y, z, visibility) for each landmark
        feature_vector = [
            [landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in landmarks
        ]
        feature_vector = np.array(feature_vector).flatten().reshape(1, -1)
        feature_vector = scaler.transform(feature_vector)  # Standardize the feature vector
        return feature_vector
    except Exception as e:
        print(f"Error during preprocessing: {e}")
        return None

# Function to predict activity from processed landmarks
def predict_activity(processed_landmarks):
    try:
        predictions = model.predict(processed_landmarks)
        predicted_class = np.argmax(predictions, axis=1)[0]
        return label_encoder.inverse_transform([predicted_class])[0]
    except Exception as e:
        print(f"Error during prediction: {e}")
        return "Prediction Error"

# Start webcam feed
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Pose estimation
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No frame received from webcam. Exiting.")
            break

        # Convert image to RGB for Mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose landmarks
        results = pose.process(image)

        # Convert back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # If pose landmarks are detected
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            landmarks = results.pose_landmarks.landmark

            # Preprocess landmarks and predict activity
            processed_landmarks = preprocess_landmarks(landmarks)
            activity = (
                predict_activity(processed_landmarks)
                if processed_landmarks is not None
                else "Processing Error"
            )
        else:
            activity = "No Person Detected"

        # Display the activity label on the video feed
        cv2.putText(image, f"Activity: {activity}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Show the video feed
        cv2.imshow('Real-time Activity Detection', image)

        # Quit with 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            print("Exiting application.")
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


# TEST 4

In [23]:
%pip install tensorflow

# Operation 4

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
import joblib

# Load the dataset
data = pd.read_csv('pose_data.csv')

# Separate features and labels
X = data.drop(columns=['Label'])
y = data['Label']

# Encode labels into integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Save label encoder for future use
joblib.dump(label_encoder, 'label_encoder.pkl')

# Standardize the feature data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Save the scaler for future use
joblib.dump(scaler, 'scaler.pkl')

# One-hot encode the labels for deep learning
y_one_hot = to_categorical(y_encoded)

# Train-test split (using stratify for balanced classes)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_one_hot, test_size=0.2, random_state=42, stratify=y_one_hot
)

# Build the deep learning model with improved architecture
model = Sequential([    
    Dense(512, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(256, activation='relu', kernel_regularizer=l2(0.0005)),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(128, activation='relu', kernel_regularizer=l2(0.0005)),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.0005)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(y_one_hot.shape[1], activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Lower learning rate for more stability
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for learning rate adjustment and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)  # Longer patience

# Train the model with additional steps
history = model.fit(
    X_train, y_train,
    epochs=1000,  # Increased epochs to allow for more training time
    batch_size=64,
    validation_split=0.2,
    callbacks=[reduce_lr, early_stop],
    shuffle=True  # Shuffle data during training
)

# Evaluate the model
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot encoded test labels back to integers

# Classification report
labels = np.unique(y_test_labels)
print("Classification Report:\n", classification_report(
    y_test_labels,
    y_pred,
    labels=labels,
    target_names=label_encoder.inverse_transform(labels)
))

# Save the entire model (architecture, weights, and optimizer state)
model.save('activity_classifier_optimized.h5')

# Save only the model weights (ensure the filename ends with `.weights.h5`)
model.save_weights('model_weights.weights.h5')

print("Model and preprocessing artifacts saved successfully!")


c:\anacondadata\envs\human\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 281ms/step - accuracy: 0.0848 - loss: 3.3573 - val_accuracy: 0.0779 - val_loss: 2.6649 - learning_rate: 5.0000e-04
Epoch 2/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.1418 - loss: 3.4628 - val_accuracy: 0.1558 - val_loss: 2.5534 - learning_rate: 5.0000e-04
Epoch 3/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1427 - loss: 3.2217 - val_accuracy: 0.4675 - val_loss: 2.4501 - learning_rate: 5.0000e-04
Epoch 4/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.1956 - loss: 2.6448 - val_accuracy: 0.5065 - val_loss: 2.3610 - learning_rate: 5.0000e-04
Epoch 5/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.2511 - loss: 2.6685 - val_accuracy: 0.5584 - val_loss: 2.2715 - learning_rate: 5.0000e-04
Epoch 6/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.3226 - loss: 2.3344 - val_accuracy: 0.5844 - val_loss: 2.1916 - learning_rate: 5.0000e-04
Epoch 7/1000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.3337

c:\anacondadata\envs\human\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\anacondadata\envs\human\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\anacondadata\envs\human\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
                    precision    recall  f1-score   support

            cough       0.80      0.80      0.80         5
         coughing       0.00      0.00      0.00         7
          falling       0.89      0.62      0.73        13
         headache       0.77      0.96      0.86        25
     heart stroke       0.73      0.69      0.71        16
      heartstroke       0.00      0.00      0.00         1
          sitting       0.67      0.93      0.78        15
         standing       1.00      0.89      0.94         9
standing normally       1.00      1.00      1.00         6

         accuracy                           0.77        97
        macro avg       0.65      0.65      0.65        97
     weighted avg       0.74      0.77      0.75        97

Model and preprocessing artifacts saved successfully!


# cam test 4

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
import tensorflow as tf

# Load the trained model, scaler, and label encoder
model = tf.keras.models.load_model('activity_classifier_optimized.h5')
scaler = joblib.load('scaler.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# Initialize Mediapipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to preprocess pose landmarks
def preprocess_landmarks(landmarks):
    try:
        feature_vector = [
            [landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in landmarks
        ]
        feature_vector = np.array(feature_vector).flatten().reshape(1, -1)
        feature_vector = scaler.transform(feature_vector)  # Standardize the feature vector
        return feature_vector
    except Exception as e:
        print(f"Error during preprocessing: {e}")
        return None

# Function to predict activity from processed landmarks
def predict_activity(processed_landmarks):
    try:
        predictions = model.predict(processed_landmarks)
        predicted_class = np.argmax(predictions, axis=1)[0]
        return label_encoder.inverse_transform([predicted_class])[0]
    except Exception as e:
        print(f"Error during prediction: {e}")
        return "Prediction Error"

# Start webcam feed
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Pose estimation
with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("No frame received from webcam. Exiting.")
            break

        # Convert image to RGB for Mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose landmarks
        results = pose.process(image)

        # Convert back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # If pose landmarks are detected
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            landmarks = results.pose_landmarks.landmark

            # Preprocess landmarks and predict activity
            processed_landmarks = preprocess_landmarks(landmarks)
            activity = (
                predict_activity(processed_landmarks)
                if processed_landmarks is not None
                else "Processing Error"
            )
        else:
            activity = "No Person Detected"

        # Display the activity label on the video feed
        cv2.putText(image, f"Activity: {activity}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Show the video feed
        cv2.imshow('Real-time Activity Detection', image)

        # Quit with 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            print("Exiting application.")
            break

# Release resources
cap.release()
cv2.destroyAllWindows()
